In [1]:
class Node:
    def __init__(self, node_type, value, left=None, right=None):
        self.node_type = node_type
        self.value = value
        self.left = left
        self.right = right

    def __repr__(self):
        left = f"Left({self.left})" if self.left else "None"
        right = f"Right({self.right})" if self.right else "None"
        return f"Node(type={self.node_type}, value={self.value}, {left}, {right})"



In [2]:
reso = "(age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')"

#these are conditions and operations e.g. condition age>30 and operator AND, OR(these will be interal node)

In [4]:
''' We need to start constructing thi the operator that is out of all the brackets, which will be the parent of all '''
''' higher the operator precedence lower it is placed'''

''' BUT if new addition is make like (existing rule ) AND (xyz), then AND WILL BE ON TOP OF EXISTING RULES OPERATORS
In the given scenario, AND is placed at the top because we are manually constructing a new AST by explicitly introducing
a new condition that is to be combined with the entire existing rule using the AND operator. This operation is different 
from how operator precedence works within a single rule.'''


operator_precedence = {
    "AND" : 1,
    "OR" : 2
}
#BRACKETS WILL BE CONSIDERED WITHIN ALGORITHM 

In [5]:
def evaluate_rule(node, user_data):
    if node.type == "operand":
        condition = node.value  
        return eval_condition(condition, user_data)

    elif node.type == "operator":
        left_result = evaluate_rule(node.left, user_data)
        right_result = evaluate_rule(node.right, user_data)

        if node.value == "AND":
            return left_result and right_result
        elif node.value == "OR":
            return left_result or right_result

def eval_condition(condition, user_data):
    # Example: condition is "age > 30"
    attribute, operator, value = parse_condition(condition)
    user_value = user_data[attribute]  # Get the user's value for "age"
    return apply_operator(user_value, operator, value)

# Helper functions for parsing and applying operators would be defined separately


In [6]:
import re

def tokenize(rule_string):

    tokens = re.findall(r'\w+\s*=\s*\'\w+\'|\w+\s*[><=]\s*\d+|\(|\)|AND|OR', rule_string)
    return tokens

def parse_condition(condition):

    condition = condition.strip()
    if '>' in condition:
        attribute, value = condition.split('>')
        return lambda data: data[attribute.strip()] > int(value.strip())
    elif '<' in condition:
        attribute, value = condition.split('<')
        return lambda data: data[attribute.strip()] < int(value.strip())
    elif '=' in condition:
        attribute, value = condition.split('=')
        value = value.strip().strip("'")
        return lambda data: data[attribute.strip()] == value

def parse_expression(tokens):
    """ Parse tokens into an AST recursively. """
    stack = []
    
    def create_node(op_type, left, right):
        return Node(node_type="operator", value=op_type, left=left, right=right)
    
    i = 0
    while i < len(tokens):
        token = tokens[i]
        
        if token == "(":
            stack.append(token)
        elif token == ")":
            # Collapse the stack into a subtree
            right = stack.pop()
            operator = stack.pop()
            left = stack.pop()
            stack.pop()  # Remove '('
            stack.append(create_node(operator, left, right))
        elif token in ["AND", "OR"]:
            stack.append(token)
        else:
            # It's a condition, create a leaf node (operand)
            stack.append(Node(node_type="operand", value=token))
        
        i += 1
    
    return stack[0]  # Return the root node of the AST

def create_rule(rule_string):

    tokens = tokenize(rule_string)
    return parse_expression(tokens)

rule = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing'))"
ast = create_rule(rule)
print(ast)



Node(type=operator, value=OR, Left(Node(type=operator, value=AND, Left(Node(type=operand, value=age > 30, None, None)), Right(Node(type=operand, value=department = 'Sales', None, None)))), Right(Node(type=operator, value=AND, Left(Node(type=operand, value=age < 25, None, None)), Right(Node(type=operand, value=department = 'Marketing', None, None)))))


In [8]:
def evaluate_condition(node, data):
    return parse_condition(node.value)(data)

def evaluate_ast(node, data):
    if node.node_type == "operand":
        return evaluate_condition(node, data)
    
    left_result = evaluate_ast(node.left, data)
    right_result = evaluate_ast(node.right, data)
    
    if node.value == "AND":
        return left_result and right_result
    elif node.value == "OR":
        return left_result or right_result

user_data_1 = {
    "age": 35,
    "department": "Sales",
    "salary": 60000,
    "experience": 3
}

user_data_2 = {
    "age": 23,
    "department": "Sales",
    "salary": 300,
    "experience": 1
}


print(evaluate_ast(ast, user_data_1))  
print(evaluate_ast(ast, user_data_2)) 


True
False


In [10]:
def combine_rules(asts, operator):
    if not asts:
        return None
    if len(asts) == 1:
        return asts[0]
    
    combined_ast = asts[0]
    for ast in asts[1:]:
        combined_ast = Node(node_type="operator", value=operator, left=combined_ast, right=ast)
    
    return combined_ast

rule_1 = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing'))"
rule_2 = "((salary > 50000 OR experience > 5))"

ast_1 = create_rule(rule_1)
ast_2 = create_rule(rule_2)

combined_ast = combine_rules([ast_1, ast_2], "AND")
print(evaluate_ast(combined_ast, user_data_1))  
print(evaluate_ast(combined_ast, user_data_2))  


IndexError: pop from empty list

In [11]:
import json

def ast_to_dict(node):
    if node is None:
        return None
    return {
        "type": node.node_type,
        "value": node.value,
        "left": ast_to_dict(node.left),
        "right": ast_to_dict(node.right)
    }

def dict_to_ast(d):
    if d is None:
        return None
    return Node(node_type=d['type'], value=d['value'], left=dict_to_ast(d['left']), right=dict_to_ast(d['right']))

# Example:
ast_json = json.dumps(ast_to_dict(ast))  
print(ast_json)

restored_ast = dict_to_ast(json.loads(ast_json))  
print(restored_ast)


{"type": "operator", "value": "OR", "left": {"type": "operator", "value": "AND", "left": {"type": "operand", "value": "age > 30", "left": null, "right": null}, "right": {"type": "operand", "value": "department = 'Sales'", "left": null, "right": null}}, "right": {"type": "operator", "value": "AND", "left": {"type": "operand", "value": "age < 25", "left": null, "right": null}, "right": {"type": "operand", "value": "department = 'Marketing'", "left": null, "right": null}}}
Node(type=operator, value=OR, Left(Node(type=operator, value=AND, Left(Node(type=operand, value=age > 30, None, None)), Right(Node(type=operand, value=department = 'Sales', None, None)))), Right(Node(type=operator, value=AND, Left(Node(type=operand, value=age < 25, None, None)), Right(Node(type=operand, value=department = 'Marketing', None, None)))))


In [12]:
from flask_cors import CORS
from datetime import datetime, timedelta
from collections import deque

from utils.create_json import ast_to_dict, dict_to_ast, save_ast_to_json

import os
import requests
from logic.parsing import tokenize, parse_expression
from flask import Flask, jsonify, request, send_from_directory, render_template


def get_rule():
    rule = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing'))"

    # restored_ast = dict_to_ast(json.loads(ast_json)) 
    path_to_json = r'D:\AST_project\src\files\ast_dict.json'
    tokenized_rule = tokenize(rule)
    rule = parse_expression(tokenized_rule)
    ast_dict = ast_to_dict(rule)
    save_ast_to_json(rule,path_to_json )
    return ast_dict

diii = get_rule()


An error occurred while saving AST to JSON: [Errno 2] No such file or directory: 'D:\\AST_project\\src\\files\\ast_dict.json'


In [13]:
print(diii)

{'type': 'operator', 'value': 'OR', 'left': {'type': 'operator', 'value': 'AND', 'left': {'type': 'operand', 'value': 'age > 30', 'left': None, 'right': None}, 'right': {'type': 'operand', 'value': "department = 'Sales'", 'left': None, 'right': None}}, 'right': {'type': 'operator', 'value': 'AND', 'left': {'type': 'operand', 'value': 'age < 25', 'left': None, 'right': None}, 'right': {'type': 'operand', 'value': "department = 'Marketing'", 'left': None, 'right': None}}}


In [15]:
from utils.create_json import load_ast_from_json, dict_to_ast
from utils.collect_operands import collect_operands, collect_operands_for_user_data
rule = load_ast_from_json(r'files/ast_dict.json')
print(rule)


print(collect_operands(rule))
collect_operands_for_user_data(rule)


AST loaded successfully from 'files/ast_dict.json'.
Node(type=operator, value=OR, Left(Node(type=operator, value=AND, Left(Node(type=operand, value=age > 30, None, None)), Right(Node(type=operand, value=department = 'Sales', None, None)))), Right(Node(type=operator, value=AND, Left(Node(type=operand, value=age < 25, None, None)), Right(Node(type=operand, value=department = 'Marketing', None, None)))))
['age > 30', "department = 'Sales'", 'age < 25', "department = 'Marketing'"]


['age', 'department']